In [53]:
import pandas as pd
import numpy as np 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import nltk
nltk.download('punkt')#
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')#
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') #
import sys  
!{sys.executable} -m pip install pyspellchecker 
from spellchecker import SpellChecker 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cyrill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cyrill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Cyrill\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cyrill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
train = pd.read_csv('test.csv',encoding ="utf-8" )
#pd.set_option('display.max_colwidth', -1)

In [119]:
#train.drop(["id"], axis=1, inplace=True)

In [120]:
train["text"]=train["text"].str.lower()

In [121]:
trainS = train
trainS.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0                  just happened a terrible car crash  
1   heard about #earthquake is different cities, s...  
2   there is a forest fire at spot pond, geese are...  
3            apocalypse lighting. #spokane #wildfires  
4       typhoon soudelor kills 28 in china and taiwan  
5                  we're shaking...it's an earthquake  
6   they'd probably still show more life than arse...  
7                                   hey! how are you?  
8                                    what a nice hat?  
9                                           fuck off!  
10                              no i don't like cold!  
11                         nooooooooo! don't do that!  
12                             no don't tell me that!  
13                                          what if?!  
14                                           awesome!  
15  birmingham wholesale market is ablaze bbc news...  
16  @sunkxssedharry will you wear shorts for race ...  
17  #previouslyondoyintv: toke makinwaûªs marriag...  
18  check these out: http://t.co/roi2nsmejj http:/...  
19  psa: iûªm splitting my personalities.\r\n\r\n...

In [122]:
Contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"gonna": "going to",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",


}

"""
       "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"

"""

'\n       "$" : " dollar ",\n    "€" : " euro ",\n    "4ao" : "for adults only",\n    "a.m" : "before midday",\n    "a3" : "anytime anywhere anyplace",\n    "aamof" : "as a matter of fact",\n    "acct" : "account",\n    "adih" : "another day in hell",\n    "afaic" : "as far as i am concerned",\n    "afaict" : "as far as i can tell",\n    "afaik" : "as far as i know",\n    "afair" : "as far as i remember",\n    "afk" : "away from keyboard",\n    "app" : "application",\n    "approx" : "approximately",\n    "apps" : "applications",\n    "asap" : "as soon as possible",\n    "asl" : "age, sex, location",\n    "atk" : "at the keyboard",\n    "ave." : "avenue",\n    "aymm" : "are you my mother",\n    "ayor" : "at your own risk", \n    "b&b" : "bed and breakfast",\n    "b+b" : "bed and breakfast",\n    "b.c" : "before christ",\n    "b2b" : "business to business",\n    "b2c" : "business to customer",\n    "b4" : "before",\n    "b4n" : "bye for now",\n    "b@u" : "back at you",\n    "bae" : "bef

In [123]:
 def decontraction(total_text):
        
        for i in Contractions:
            total_text = total_text.replace( i, Contractions.get(i))
        
        return total_text

In [124]:
trainS['text'] = trainS['text'].map(decontraction)
trainS.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0                  just happened a terrible car crash  
1   heard about #earthquake is different cities, s...  
2   there is a forest fire at spot pond, geese are...  
3            apocalypse lighting. #spokane #wildfires  
4       typhoon soudelor kills 28 in china and taiwan  
5                we are shaking...it is an earthquake  
6   they would probably still show more life than ...  
7                                   hey! how are you?  
8                                    what a nice hat?  
9                                           fuck off!  
10                             no i do not like cold!  
11                        nooooooooo! do not do that!  
12                            no do not tell me that!  
13                                          what if?!  
14                                           awesome!  
15  birmingham wholesale market is ablaze bbc news...  
16  @sunkxssedharry will you wear shorts for race ...  
17  #previouslyondoyintv: toke makinwaûªs marriag...  
18  check these out: http://t.co/roi2nsmejj http:/...  
19  psa: iûªm splitting my personalities.\r\n\r\n...

In [125]:
def data_text_preprocess(total_text):
        
        total_text = re.sub('http\S+',' ', str(total_text))
        total_test = re.sub('https\S+',' ', str(total_text))
        total_text = re.sub('\@\S+',' ', str(total_text))
        total_text = re.sub('\#',' ', str(total_text))
        total_text = re.sub(r" \B'\b|\b'\B", " ", str(total_text))
        total_text = re.sub('[^a-zA-Z\n]', ' ', str(total_text))
       
        
       
        return total_text

In [126]:
trainS['text'] = trainS['text'].map(data_text_preprocess)
trainS.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0                  just happened a terrible car crash  
1   heard about  earthquake is different cities  s...  
2   there is a forest fire at spot pond  geese are...  
3            apocalypse lighting   spokane  wildfires  
4       typhoon soudelor kills    in china and taiwan  
5                we are shaking   it is an earthquake  
6   they would probably still show more life than ...  
7                                   hey  how are you   
8                                    what a nice hat   
9                                           fuck off   
10                             no i do not like cold   
11                        nooooooooo  do not do that   
12                            no do not tell me that   
13                                          what if    
14                                           awesome   
15  birmingham wholesale market is ablaze bbc news...  
16             will you wear shorts for race ablaze    
17   previouslyondoyintv  toke makinwa   s marriag...  
18                     check these out           nsfw  
19  psa  i   m splitting my personalities  \n \n  ...

In [127]:
def get_wordnet_pos(word):
   
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [128]:
lemmatizer = WordNetLemmatizer()

def lemmatization1(total_text):
    total_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(total_text)]

    return total_text

In [129]:
trainS['text'] = trainS['text'].map(lemmatization1)
trainS.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0             [just, happen, a, terrible, car, crash]  
1   [heard, about, earthquake, be, different, city...  
2   [there, be, a, forest, fire, at, spot, pond, g...  
3              [apocalypse, light, spokane, wildfire]  
4   [typhoon, soudelor, kill, in, china, and, taiwan]  
5             [we, be, shake, it, be, an, earthquake]  
6   [they, would, probably, still, show, more, lif...  
7                                 [hey, how, be, you]  
8                                [what, a, nice, hat]  
9                                         [fuck, off]  
10                       [no, i, do, not, like, cold]  
11                    [nooooooooo, do, not, do, that]  
12                      [no, do, not, tell, me, that]  
13                                         [what, if]  
14                                          [awesome]  
15  [birmingham, wholesale, market, be, ablaze, bb...  
16        [will, you, wear, short, for, race, ablaze]  
17  [previouslyondoyintv, toke, makinwa, s, marria...  
18                          [check, these, out, nsfw]  
19  [psa, i, m, splitting, my, personality, techie...

In [130]:
stop_words = set(stopwords.words('english'))


In [131]:
def remove_stopwords(total_text):
    
    filtered_word_list = total_text[:]
    for word in total_text:
        if word in stop_words: 
            filtered_word_list.remove(word)
    
    #filtered_sentence = [w for w in total_text if not w in stop_words] 
    #filtered_sentence= []
    
    #for w in total_text: 
      #  if w not in stop_words: 
       #     filtered_sentence.append(w) 

    return filtered_word_list

In [132]:
trainS['text'] = trainS['text'].map(remove_stopwords)
trainSR = trainS
trainSR.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0                      [happen, terrible, car, crash]  
1   [heard, earthquake, different, city, stay, saf...  
2   [forest, fire, spot, pond, geese, flee, across...  
3              [apocalypse, light, spokane, wildfire]  
4            [typhoon, soudelor, kill, china, taiwan]  
5                                 [shake, earthquake]  
6   [would, probably, still, show, life, arsenal, ...  
7                                               [hey]  
8                                         [nice, hat]  
9                                              [fuck]  
10                                       [like, cold]  
11                                       [nooooooooo]  
12                                             [tell]  
13                                                 []  
14                                          [awesome]  
15  [birmingham, wholesale, market, ablaze, bbc, n...  
16                        [wear, short, race, ablaze]  
17  [previouslyondoyintv, toke, makinwa, marriage,...  
18                                      [check, nsfw]  
19  [psa, splitting, personality, techie, follow, ...

In [133]:
def removesinglelettertoken(total_text):
    emptylist=[]
    for w in total_text:
        if len(w)>1:
            emptylist.append(w)
    return emptylist

In [134]:
trainSR['text'] = trainSR['text'].map(removesinglelettertoken)

trainSR.head(20)

id keyword                        location  \
0    0     NaN                             NaN   
1    2     NaN                             NaN   
2    3     NaN                             NaN   
3    9     NaN                             NaN   
4   11     NaN                             NaN   
5   12     NaN                             NaN   
6   21     NaN                             NaN   
7   22     NaN                             NaN   
8   27     NaN                             NaN   
9   29     NaN                             NaN   
10  30     NaN                             NaN   
11  35     NaN                             NaN   
12  42     NaN                             NaN   
13  43     NaN                             NaN   
14  45     NaN                             NaN   
15  46  ablaze                          London   
16  47  ablaze  Niall's place | SAF 12 SQUAD |   
17  51  ablaze                         NIGERIA   
18  58  ablaze                  Live On Webcam   
19  60  ablaze        Los Angeles, Califnordia   

                                                 text  
0                      [happen, terrible, car, crash]  
1   [heard, earthquake, different, city, stay, saf...  
2   [forest, fire, spot, pond, geese, flee, across...  
3              [apocalypse, light, spokane, wildfire]  
4            [typhoon, soudelor, kill, china, taiwan]  
5                                 [shake, earthquake]  
6   [would, probably, still, show, life, arsenal, ...  
7                                               [hey]  
8                                         [nice, hat]  
9                                              [fuck]  
10                                       [like, cold]  
11                                       [nooooooooo]  
12                                             [tell]  
13                                                 []  
14                                          [awesome]  
15  [birmingham, wholesale, market, ablaze, bbc, n...  
16                        [wear, short, race, ablaze]  
17  [previouslyondoyintv, toke, makinwa, marriage,...  
18                                      [check, nsfw]  
19  [psa, splitting, personality, techie, follow, ...

In [109]:
spell = SpellChecker()

def spellchecker(total_text):
    for w in total_text: 
        spell.correction(w) 
    return total_text

In [110]:
def reconcatenate(total_text):
    TOTAL_text=''
    for w in total_text:
        TOTAL_text=TOTAL_text + w + ' '
    return TOTAL_text
        

In [111]:
trainSR['text']=trainSR['text'].map(reconcatenate)

In [112]:
#trainSR['text'] = trainSR['text'].map(spellchecker)

#trainSR.head(20)

In [113]:
trainSR['mean'] = trainSR.groupby('text')['target'].transform(np.mean)

#drop duplicates that contain contradictions in the target (mean of target is !=0 or !=1)
TRAINsr=trainSR[(trainSR['mean']==0) | (trainSR['mean']==1)]
#keep only one entry of perfect duplicates
TRAINsr.drop_duplicates(subset ="text", keep = 'first', inplace = True)
TRAINsr

C:\Users\Cyrill\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7601  10859     NaN      NaN   
7602  10860     NaN      NaN   
7603  10862     NaN      NaN   
7605  10864     NaN      NaN   
7606  10866     NaN      NaN   

                                                   text  target  mean  
0             deed reason earthquake may allah forgive        1   1.0  
1                forest fire near la ronge sask canada        1   1.0  
2     resident ask shelter place notify officer evac...       1   1.0  
3     people receive wildfire evacuation order calif...       1   1.0  
4     get sent photo ruby alaska smoke wildfire pour...       1   1.0  
...                                                 ...     ...   ...  
7601  break la refugio oil spill may costlier big pr...       1   1.0  
7602                      siren go forney tornado warn        1   1.0  
7603  official say quarantine place alabama home pos...       1   1.0  
7605  flip side walmart bomb everyone evacuate stay ...       1   1.0  
7606  suicide bomber kill saudi security site mosque...       1   1.0  

[6675 rows x 6 columns]

In [114]:
def retokenize(total_text):
    nltk_tokens = nltk.word_tokenize(total_text)
    return nltk_tokens

In [115]:
TRAINsr['text']=TRAINsr['text'].map(retokenize)

C:\Users\Cyrill\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
TRAINsr

id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7601  10859     NaN      NaN   
7602  10860     NaN      NaN   
7603  10862     NaN      NaN   
7605  10864     NaN      NaN   
7606  10866     NaN      NaN   

                                                   text  target  mean  
0       [deed, reason, earthquake, may, allah, forgive]       1   1.0  
1         [forest, fire, near, la, ronge, sask, canada]       1   1.0  
2     [resident, ask, shelter, place, notify, office...       1   1.0  
3     [people, receive, wildfire, evacuation, order,...       1   1.0  
4     [get, sent, photo, ruby, alaska, smoke, wildfi...       1   1.0  
...                                                 ...     ...   ...  
7601  [break, la, refugio, oil, spill, may, costlier...       1   1.0  
7602                 [siren, go, forney, tornado, warn]       1   1.0  
7603  [official, say, quarantine, place, alabama, ho...       1   1.0  
7605  [flip, side, walmart, bomb, everyone, evacuate...       1   1.0  
7606  [suicide, bomber, kill, saudi, security, site,...       1   1.0  

[6675 rows x 6 columns]

In [135]:
# Save to csv
trainSR.to_csv("submitclean.csv", index=False, encoding = 'utf-8')